In [1]:
from financepy.models.finite_difference_PSOR import black_scholes_fd_PSOR
from financepy.models.finite_difference import option_payoff
from financepy.products.equity import OptionTypes, Date
from financepy.utils.global_vars import gDaysInYear

from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

####################################################################
# FINANCEPY BETA Version 0.270 - This build:  26 Feb 2023 at 19:12 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



In [2]:
expiry_date = Date(1, 7, 2015)
strike_price = 100.0
option_type = OptionTypes.EUROPEAN_CALL

valuation_date = Date(1, 1, 2015)
stock_price = 100
volatility = 0.30
risk_free_rate = 0.05
dividend_yield = 0.01
num_std = 5
num_samples = 2000

# Time to contract expiry in years
time_to_expiry = (expiry_date - valuation_date) / gDaysInYear


In [3]:
res = black_scholes_fd_PSOR(spot_price=stock_price, volatility=volatility,
                                    time_to_expiry=time_to_expiry,
                                    strike_price=100.0, risk_free_rate=risk_free_rate,
                                    dividend_yield=dividend_yield, digital=0,
                                    option_type=option_type, smooth=0, theta=0.5, wind=0,
                                    num_std=5, num_time_steps=None, num_samples=num_samples)

In [4]:
print(f"Value of option is ${res}")

Value of option is $9.302028751692234
